# TEST - Run the preprocessing step on AWS Batch
This notebook will run preprocess.py on the AWS Batch container using the default API.

#### Dataset
We will use the census dataset from `sagemaker-examples` for this demo. If you wish to test with another dataset, you will need to modify the logic within preprocess.py.



In [1]:
import pandas as pd
import utils
import boto3
import json
import sagemaker
import uuid

In [2]:
bucket = 'sagemaker-on-aws-batch-643551339491-ap-southeast-1'
region = sagemaker.Session().boto_region_name
account = boto3.client('sts').get_caller_identity().get('Account')

# PRINT
print('region:', region)
print('bucket:', bucket)
print('account', account)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


region: ap-southeast-1
bucket: sagemaker-on-aws-batch-643551339491-ap-southeast-1
account 643551339491


# Download dataset locally

In [3]:
utils.mkpath_if_not_exist('data')

In [4]:
s3 = boto3.client("s3")
s3.download_file(
    "sagemaker-sample-data-{}".format(region),
    "processing/census/census-income.csv",
    "data/census-income.csv",
)
df = pd.read_csv("data/census-income.csv")
df.to_csv("data/dataset.csv")
df.head()

,age,class of worker,detailed industry recode,detailed occupation recode,education,wage per hour,enroll in edu inst last wk,marital stat,major industry code,major occupation code,...,country of birth father,country of birth mother,country of birth self,citizenship,own business or self employed,fill inc questionnaire for veteran's admin,veterans benefits,weeks worked in year,year,income
0,73,Not in universe,0,0,High school graduate,0,Not in universe,Widowed,Not in universe or children,Not in universe,...,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,2,0,95,- 50000.
1,58,Self-employed-not incorporated,4,34,Some college but no degree,0,Not in universe,Divorced,Construction,Precision production craft & repair,...,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,2,52,94,- 50000.
2,18,Not in universe,0,0,10th grade,0,High school,Never married,Not in universe or children,Not in universe,...,Vietnam,Vietnam,Vietnam,Foreign born- Not a citizen of U S,0,Not in universe,2,0,95,- 50000.
3,9,Not in universe,0,0,Children,0,Not in universe,Never married,Not in universe or children,Not in universe,...,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,0,0,94,- 50000.
4,10,Not in universe,0,0,Children,0,Not in universe,Never married,Not in universe or children,Not in universe,...,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,0,0,94,- 50000.


# Upload Input data and code to S3

In [5]:
s3.upload_file('data/dataset.csv',bucket,'data/sample/census/dataset.csv', )

In [17]:
s3.upload_file('preprocess.py',bucket,'code-repo/sagemaker-process-code/preprocess.py')
s3.upload_file('sagemaker_entry_point.py',bucket,'code-repo/sagemaker-process-code/sagemaker_entry_point.py')
s3.upload_file('utils.py',bucket,'code-repo/sagemaker-process-code/utils.py')

# Run AWS Batch Job

In [18]:
def create_unique_job_name(base_job_name, max_length=60):
    import uuid
    from datetime import datetime
    datetime_uuid=datetime.strftime(datetime.now(), "%Y%m%d%H%S")
    return f'{base_job_name}-{datetime_uuid}'[:max_length]

def get_account_details():
    account = boto3.client('sts').get_caller_identity().get('Account')
    region = boto3.session.Session().region_name
    env= 'dev' if account =='643551339491' else 'prod'
    
    return account,region,env

def run_process_job(train_test_split,
                    validation_flag,
                    job_queue,
                    job_definition,
                    batch_platform='arm64'
                   ):
    
    batch = boto3.client('batch')
    
    account,region,env = get_account_details()
    INPUT_BUCKET= 'sagemaker-on-aws-batch-643551339491-ap-southeast-1'
    CODE_BUCKET ='sagemaker-on-aws-batch-643551339491-ap-southeast-1'
    OUTPUT_BUCKET = 'sagemaker-on-aws-batch-643551339491-ap-southeast-1'
    PREPROCESING_ENTRYPOINT = f"s3://{CODE_BUCKET}/code-repo/sagemaker-process-code/sagemaker_entry_point.py"

    #AWS BATCH PARAMETERS
    job_name=create_unique_job_name(base_job_name=f'data-process-aws-batch-run'.replace('_','-'),max_length=60)
        
    parameters = {
    'train-test-split' : train_test_split,
    'validataion-flag' : validation_flag,
    'env': env,
    'entry-point': PREPROCESING_ENTRYPOINT,
    'input-bucket': INPUT_BUCKET,
    'output-bucket': OUTPUT_BUCKET,
    'code-bucket': CODE_BUCKET,
    }

    try:
        # Submit a Batch Job
        response = batch.submit_job(jobQueue=job_queue, 
                                    jobName=job_name,
                                    jobDefinition=job_definition, 
                                    parameters=parameters)
        # Log response from AWS Batch
        print(f"Job submitted for jobName={job_name} for parameters={parameters}, Response: " + json.dumps(response, indent=2))
        # Return the jobId
        jobId = response['jobId']
        return jobId
    except Exception as e:
        print(e)
        message = f'Error submitting Batch Job for jobName={job_name} for parameters={parameters}'
        print(message)
        raise Exception(message)

In [19]:
#ARM 64 JOB QUEUE
job_queue=f'arn:aws:batch:{region}:{account}:job-queue/aws-dev-aws-batch-arm64-job-queue'
job_definition=f'arn:aws:batch:{region}:{account}:job-definition/aws-dev-aws-batch-arm64-job-definition'
    
job_id = run_process_job(train_test_split="0.2",
                         validation_flag="True",
                         job_queue=job_queue,
                         job_definition=job_definition,
                        )
    

Job submitted for jobName=data-process-aws-batch-run-True-941558d6-be41-4b27-93b7-24c4 for parameters={'train-test-split': '0.2', 'validataion-flag': 'True', 'env': 'prod', 'entry-point': 's3://sagemaker-on-aws-batch-643551339491-ap-southeast-1/code-repo/sagemaker-process-code/sagemaker_entry_point.py', 'input-bucket': 'sagemaker-on-aws-batch-643551339491-ap-southeast-1', 'output-bucket': 'sagemaker-on-aws-batch-643551339491-ap-southeast-1', 'code-bucket': 'sagemaker-on-aws-batch-643551339491-ap-southeast-1'}, Response: {
  "ResponseMetadata": {
    "RequestId": "11460c7f-7db3-42a1-ba85-502c86c9d913",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Mon, 30 May 2022 00:55:01 GMT",
      "content-type": "application/json",
      "content-length": "215",
      "connection": "keep-alive",
      "x-amzn-requestid": "11460c7f-7db3-42a1-ba85-502c86c9d913",
      "access-control-allow-origin": "*",
      "x-amz-apigw-id": "S6hb4EZNyQ0Fhrg=",
      "access-control-expose-headers"

------